In [ ]:
import socket 
import threading 

#encryption and decryption
def encrypt_message(message, key=3):
    encrypted = ''.join(chr ((ord(char) - 32 + key) % 95 + 32) if char.isprintable() else char for char in message)
    return encrypted
def decrypt_message (message, keys=3):
    decrypted = ''.join(chr((ord(char)- 32 - key) % 95 + 32)if char.isprintable() else char for char in message)
    return decrypted

def receive_messages(client_socket):
    while True:
        try:
            data, _ = client_socket.recvfrom(2048)
            decrypted_message = decrypt_message(data.decode())
            print(decrypted_message)
        except:
            break
        
#setup udp client
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)

#authentication
username = input ("Enter your username: ")
password = input("Enter your password: ")
auth_message = f"{username}: {password}"
client_socket.sendto(auth_message.encode(), server_address)

#response to authentication:
data,_ = client_socket.recvfrom(2048)
response = data.decode()

if response == "AUTH_SUCCESS":
    print("Authentication is successful! You may enter the chat")
else:
    print("Authentication has failed. Connection ended.")
    client_socket.close()
    exit()
    
#start thread - listen messages
receive_thread = threading.Thread(target=receive_messages, args=(client_socket))
receive_thread.daemon = True
receive_thread.start()

#send the messages
print("Type the message (or 'quit' to leave):")
while True:
    message = input()
    if message.lower ()== 'exit':
        print("You have exited the chat.")
        break
    encrypted_message = encrypt_message(message)
    client_socket.sendto(encrypted_message.encod(), server_address)
    
    
client_socket.close() 